 --- Extracción de Datos de Artistas y Enriquecimiento con YouTube API ---

In [66]:
# Celda 1: Importar librerías
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import logging
import time
import numpy as np
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from IPython.display import display
import warnings

In [67]:
# Celda 2: Configuración Inicial
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)
warnings.filterwarnings('ignore', category=FutureWarning) # Para silenciar advertencias de pandas/seaborn si las hubiera

In [68]:
# Celda 3: Cargar Variables de Entorno y Definir Constantes
logging.info("Cargando variables de entorno...")
dotenv_path = '/home/nicolas/Escritorio/workshops/workshop_2/env/.env' # <-- CONFIRMA ESTA RUTA
load_dotenv(dotenv_path=dotenv_path)

# --- Credenciales de Base de Datos ---
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_NAME = os.getenv('POSTGRES_DB')
TABLE_GRAMMY = 'the_grammy_awards_clean'
TABLE_SPOTIFY = 'spotify_dataset_clean' # Asegúrate que sea la tabla limpia

# --- Credenciales de YouTube API (OAuth 2.0) ---
YOUTUBE_CLIENT_ID = os.getenv("YOUTUBE_CLIENT_ID")
YOUTUBE_CLIENT_SECRET = os.getenv("YOUTUBE_CLIENT_SECRET")
YOUTUBE_PROJECT_ID = os.getenv("YOUTUBE_PROJECT_ID")
YOUTUBE_AUTH_URI = os.getenv("YOUTUBE_AUTH_URI")
YOUTUBE_TOKEN_URI = os.getenv("YOUTUBE_TOKEN_URI")
YOUTUBE_REDIRECT_URIS= os.getenv("YOUTUBE_REDIRECT_URIS")

# --- Constantes de la API ---
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
YOUTUBE_SCOPES = ["https://www.googleapis.com/auth/youtube.readonly"]

# --- Archivo de Progreso ---
PROGRESS_CSV_PATH = '/home/nicolas/Escritorio/workshops/workshop_2/data/youtube_stats.csv' # <-- RUTA PARA GUARDAR PROGRESO

# Verificar variables de entorno
# ... (código de verificación igual que antes) ...
if not all([POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_HOST, POSTGRES_PORT, POSTGRES_NAME]):
    logging.error("Faltan variables de entorno para la base de datos en " + dotenv_path)
    raise ValueError("Variables de entorno de DB incompletas.")
if not YOUTUBE_CLIENT_ID or not YOUTUBE_CLIENT_SECRET:
    logging.error("Faltan YOUTUBE_CLIENT_ID y/o YOUTUBE_CLIENT_SECRET en " + dotenv_path)
    raise ValueError("Variables de entorno de YouTube incompletas.")

logging.info("Variables de entorno cargadas.")

# Verificar variables de entorno
if not all([POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_HOST, POSTGRES_PORT, POSTGRES_NAME]):
    logging.error("Faltan variables de entorno para la base de datos en " + dotenv_path)
    raise ValueError("Variables de entorno de DB incompletas.")
if not YOUTUBE_CLIENT_ID or not YOUTUBE_CLIENT_SECRET:
    logging.error("Faltan YOUTUBE_CLIENT_ID y/o YOUTUBE_CLIENT_SECRET en " + dotenv_path)
    raise ValueError("Variables de entorno de YouTube incompletas.")

logging.info("Variables de entorno cargadas.")

2025-04-11 02:55:54,829 - INFO - Cargando variables de entorno...
2025-04-11 02:55:54,833 - INFO - Variables de entorno cargadas.
2025-04-11 02:55:54,833 - INFO - Variables de entorno cargadas.


In [69]:
# Celda 4: Conectar a PostgreSQL
engine = None
try:
    logging.info(f"Creando motor SQLAlchemy para la base de datos '{POSTGRES_NAME}'...")
    db_url = f'postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_NAME}'
    engine = create_engine(db_url)
    logging.info(f"Motor SQLAlchemy creado exitosamente.")
except Exception as e:
    logging.error(f"Error al crear el motor SQLAlchemy: {e}")
    raise


2025-04-11 02:55:54,839 - INFO - Creando motor SQLAlchemy para la base de datos 'artists'...
2025-04-11 02:55:54,840 - INFO - Motor SQLAlchemy creado exitosamente.


In [70]:
# Celda 5: Extraer Nombres de Artistas Únicos de PostgreSQL
df_artists_spotify = None
df_artists_grammy = None
unique_artists_full_list = []

if engine:
    try:
        logging.info(f"Extrayendo artistas únicos de '{TABLE_SPOTIFY}'...")
        query_spotify = f'SELECT DISTINCT artists AS artist_name FROM "{TABLE_SPOTIFY}" WHERE artists IS NOT NULL'
        df_artists_spotify = pd.read_sql_query(query_spotify, con=engine)
        logging.info(f"Artistas extraídos de Spotify: {len(df_artists_spotify)}")

        logging.info(f"Extrayendo artistas únicos de '{TABLE_GRAMMY}'...")
        query_grammy = f'SELECT DISTINCT artist AS artist_name FROM "{TABLE_GRAMMY}" WHERE artist IS NOT NULL AND artist != \'No Especificado\'' # Excluir placeholder
        df_artists_grammy = pd.read_sql_query(query_grammy, con=engine)
        logging.info(f"Artistas extraídos de Grammys: {len(df_artists_grammy)}")

        # Renombrar columnas para poder concatenar ---
        df_artists_spotify.rename(columns={'artists': 'artist_name'}, inplace=True)
        df_artists_grammy.rename(columns={'artist': 'artist_name'}, inplace=True)
        # Concatenar los dataframes
        df_combined_artists = pd.concat([df_artists_spotify, df_artists_grammy], ignore_index=True)
        # Obtener valores únicos de la columna combinada 'artist_name'
        unique_artists_series = df_combined_artists['artist_name'].drop_duplicates()
        # Filtrar nombres comunes no útiles o placeholders
        excluded_names = ['Various Artists', '(Various Artists)', 'No Especificado', None, np.nan]
        # Asegurarse que la serie sea de strings antes de isin
        unique_artists_series = unique_artists_series.astype(str)
        unique_artists_series = unique_artists_series[~unique_artists_series.isin(excluded_names)]
        # Limpiar espacios extra por si acaso
        unique_artists_full_list = unique_artists_series.str.strip().unique().tolist()
        # Eliminar cadenas vacías si las hubiera después del strip
        unique_artists_full_list = [artist for artist in unique_artists_full_list if artist]

        logging.info(f"Lista completa de artistas únicos a procesar: {len(unique_artists_full_list)}")
        # print(f"Muestra de artistas únicos: {unique_artists_full_list[:10]}") # Mostrar muestra

    except Exception as e:
        logging.error(f"Error al extraer artistas de la base de datos: {e}")
        unique_artists_full_list = []

else:
    logging.error("Engine no disponible. No se pueden extraer artistas.")

2025-04-11 02:55:54,851 - INFO - Extrayendo artistas únicos de 'spotify_dataset_clean'...
2025-04-11 02:55:54,958 - INFO - Artistas extraídos de Spotify: 31437
2025-04-11 02:55:54,959 - INFO - Extrayendo artistas únicos de 'the_grammy_awards_clean'...
2025-04-11 02:55:54,963 - INFO - Artistas extraídos de Grammys: 1658
2025-04-11 02:55:54,979 - INFO - Lista completa de artistas únicos a procesar: 32690


In [71]:
# Celda 6: Cargar Progreso Anterior y Determinar Artistas Pendientes
df_processed_so_far = pd.DataFrame() # DataFrame para datos ya procesados
artists_already_processed = set()

try:
    logging.info(f"Intentando cargar progreso anterior desde: {PROGRESS_CSV_PATH}")
    df_processed_so_far = pd.read_csv(PROGRESS_CSV_PATH)
    # Asegurar que la columna 'artist_query' exista y no tenga nulos antes de crear el set
    if 'artist_query' in df_processed_so_far.columns:
        artists_already_processed = set(df_processed_so_far['artist_query'].dropna().unique())
        logging.info(f"Progreso cargado. {len(artists_already_processed)} artistas ya procesados.")
    else:
        logging.warning(f"El archivo CSV de progreso no contiene la columna 'artist_query'. Se procesarán todos.")
        df_processed_so_far = pd.DataFrame() # Resetear si la columna clave no está

except FileNotFoundError:
    logging.info("No se encontró archivo de progreso previo. Se iniciará desde cero.")
except Exception as e:
    logging.error(f"Error al cargar el archivo de progreso CSV: {e}. Se iniciará desde cero.")
    df_processed_so_far = pd.DataFrame() # Resetear en caso de error de lectura

# Filtrar la lista completa para obtener solo los pendientes
artists_to_process = [artist for artist in unique_artists_full_list if artist not in artists_already_processed]
logging.info(f"Artistas pendientes de procesar en esta ejecución: {len(artists_to_process)}")

2025-04-11 02:55:54,987 - INFO - Intentando cargar progreso anterior desde: /home/nicolas/Escritorio/workshops/workshop_2/data/youtube_stats.csv
2025-04-11 02:55:54,991 - INFO - Progreso cargado. 97 artistas ya procesados.
2025-04-11 02:55:54,993 - INFO - Artistas pendientes de procesar en esta ejecución: 32593


In [72]:
# Celda 7: Autenticación con YouTube API (OAuth 2.0) - Solo si hay pendientes
youtube = None
if artists_to_process: # Solo autenticar si hay trabajo que hacer
    logging.info("Iniciando autenticación con YouTube API...")
    try:
        os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
        redirect_uris = [uri.strip() for uri in YOUTUBE_REDIRECT_URIS.split(',') if uri.strip()]
        client_config = { "installed": { #... (resto de la config igual que antes) ...
                 "client_id": YOUTUBE_CLIENT_ID, "client_secret": YOUTUBE_CLIENT_SECRET,
                 "project_id": YOUTUBE_PROJECT_ID, "auth_uri": YOUTUBE_AUTH_URI,
                 "token_uri": YOUTUBE_TOKEN_URI,
                 "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
                 "redirect_uris": redirect_uris }}

        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_config(client_config, YOUTUBE_SCOPES)
        credentials = flow.run_local_server(port=0)
        youtube = googleapiclient.discovery.build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, credentials=credentials)
        logging.info("Autenticación con YouTube API exitosa.")
    except Exception as e:
        logging.error(f"Error durante la autenticación con YouTube API: {e}")
        youtube = None
else:
    logging.info("No hay nuevos artistas para procesar en esta ejecución.")

2025-04-11 02:55:55,010 - INFO - Iniciando autenticación con YouTube API...


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=571313439711-2tf8le0cpicfl5bcsc98ot6g4de5b4tf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A47715%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=aLBwbVIEhSxUTpeMENp88vTqNjLcWA&access_type=offline
Abriendo en una sesión existente del navegador


2025-04-11 02:56:00,713 - INFO - "GET /?state=aLBwbVIEhSxUTpeMENp88vTqNjLcWA&code=4/0Ab_5qll_hrkfhXVkbRytJRt42JXb1wM4y-u5t82y0Uz8KMxTS3Uaf6PJwQOHZ5ThKLre_g&scope=https://www.googleapis.com/auth/youtube.readonly HTTP/1.1" 200 65
2025-04-11 02:56:01,013 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-04-11 02:56:01,020 - INFO - Autenticación con YouTube API exitosa.


In [73]:
# Celda 8: Funciones Auxiliares para llamadas a la API (Igual que antes)
# ... (Incluir aquí las definiciones de las funciones search_for_channel,
#      get_channel_stats, search_top_videos, get_video_likes
#      de la respuesta anterior, asegurándose de que incluyan logging y
#      manejo de HttpError con posibilidad de detectar quotaExceeded) ...

def search_for_channel(youtube_service, artist_name):
    """Busca un canal por nombre y devuelve el ID del primer resultado."""
    try:
        # Añadir " Topic" puede ayudar a encontrar canales oficiales de música a veces
        search_query = f"{artist_name} Topic"
        logging.info(f"  Buscando canal para: '{artist_name}' (Query: '{search_query}')...")
        request = youtube_service.search().list(
            part="snippet",
            q=search_query,
            type="channel",
            maxResults=1
        )
        response = request.execute()
        if response.get('items'):
            channel_id = response['items'][0]['id']['channelId']
            channel_title = response['items'][0]['snippet']['title']
            logging.info(f"    Canal encontrado: '{channel_title}' (ID: {channel_id})")
            return channel_id
        else:
            # Si no se encuentra con "Topic", intentar sin él
            logging.info(f"  No se encontró canal con 'Topic'. Intentando búsqueda directa para '{artist_name}'...")
            request = youtube_service.search().list(
                part="snippet",
                q=artist_name,
                type="channel",
                maxResults=1
            )
            response = request.execute()
            if response.get('items'):
                channel_id = response['items'][0]['id']['channelId']
                channel_title = response['items'][0]['snippet']['title']
                logging.info(f"    Canal encontrado: '{channel_title}' (ID: {channel_id})")
                return channel_id
            else:
                logging.warning(f"  No se encontró canal para '{artist_name}' en búsqueda directa tampoco.")
                return None
    except googleapiclient.errors.HttpError as e:
        if 'quotaExceeded' in str(e):
             raise e # Relanzar para que el bucle principal lo capture
        logging.error(f"  Error HTTP buscando canal para '{artist_name}': {e}")
        return None
    except Exception as e:
        logging.error(f"  Error inesperado buscando canal para '{artist_name}': {e}")
        return None

def get_channel_stats(youtube_service, channel_id):
    """Obtiene estadísticas de un canal por su ID."""
    if not channel_id: return None
    try:
        logging.info(f"    Obteniendo estadísticas para Channel ID: {channel_id}")
        request = youtube_service.channels().list(
            part="statistics,snippet",
            id=channel_id
        )
        response = request.execute()
        if response.get('items'):
            item = response['items'][0]
            stats = item.get('statistics', {})
            snippet = item.get('snippet', {})
            # Convertir a int, manejar posible ausencia o valor no numérico
            try: subscriber_count = int(stats.get('subscriberCount', 0))
            except (ValueError, TypeError): subscriber_count = 0
            try: view_count = int(stats.get('viewCount', 0))
            except (ValueError, TypeError): view_count = 0
            try: video_count = int(stats.get('videoCount', 0))
            except (ValueError, TypeError): video_count = 0

            return {
                'channel_title_verified': snippet.get('title'),
                'subscriber_count': subscriber_count if not stats.get('hiddenSubscriberCount', False) else -1, # Indicar si están ocultos
                'view_count': view_count,
                'video_count': video_count,
            }
        else:
            logging.warning(f"    No se encontraron estadísticas para el channel ID: {channel_id}")
            return None
    except googleapiclient.errors.HttpError as e:
        if 'quotaExceeded' in str(e): raise e # Relanzar
        logging.error(f"    Error HTTP obteniendo estadísticas para {channel_id}: {e}")
        return None
    except Exception as e:
        logging.error(f"    Error inesperado obteniendo estadísticas para {channel_id}: {e}")
        return None

def search_top_videos(youtube_service, channel_id):
    """Busca los top 10 videos musicales más vistos de un canal."""
    video_ids = []
    if not channel_id: return []
    try:
        logging.info(f"    Buscando top 10 videos musicales para Channel ID: {channel_id}")
        request = youtube_service.search().list(
            part="id",
            channelId=channel_id,
            order="viewCount",
            type="video",
            videoCategoryId="10", # Música
            maxResults=10
        )
        response = request.execute()
        for item in response.get('items', []):
            video_ids.append(item['id']['videoId'])
        logging.info(f"      Se encontraron {len(video_ids)} IDs de videos musicales top.")
        return video_ids
    except googleapiclient.errors.HttpError as e:
        if 'quotaExceeded' in str(e): raise e # Relanzar
        # Manejar caso específico donde el canal deshabilita búsqueda por categoría
        if 'videoCategoryId filter is not supported' in str(e):
             logging.warning(f"      Búsqueda por categoría musical no soportada para {channel_id}. Intentando sin categoría...")
             try:
                 request = youtube_service.search().list(
                     part="id", channelId=channel_id, order="viewCount", type="video", maxResults=10)
                 response = request.execute()
                 for item in response.get('items', []): video_ids.append(item['id']['videoId'])
                 logging.info(f"      Se encontraron {len(video_ids)} IDs de videos top (sin filtro de categoría).")
                 return video_ids
             except Exception as inner_e:
                 logging.error(f"      Error en búsqueda sin categoría para {channel_id}: {inner_e}")
                 return []
        else:
            logging.error(f"    Error HTTP buscando videos top para {channel_id}: {e}")
            return []
    except Exception as e:
        logging.error(f"    Error inesperado buscando videos top para {channel_id}: {e}")
        return []

def get_video_likes(youtube_service, video_ids_list):
    """Obtiene el conteo de likes para una lista de IDs de video."""
    total_likes = 0
    if not video_ids_list:
        logging.info("      No hay IDs de video para buscar likes.")
        return 0

    logging.info(f"    Obteniendo likes para {len(video_ids_list)} videos top...")
    batch_size = 50 # Max por llamada
    likes_found_count = 0
    processed_ids = 0
    for i in range(0, len(video_ids_list), batch_size):
        batch_ids = video_ids_list[i:i + batch_size]
        processed_ids += len(batch_ids)
        try:
            request = youtube_service.videos().list(
                part="statistics",
                id=",".join(batch_ids)
            )
            response = request.execute()
            for item in response.get('items', []):
                likes = item.get('statistics', {}).get('likeCount')
                if likes is not None:
                    try:
                        total_likes += int(likes)
                        likes_found_count += 1
                    except (ValueError, TypeError):
                         logging.warning(f"      Valor de like no numérico encontrado para video {item.get('id')}: {likes}")

        except googleapiclient.errors.HttpError as e:
            if 'quotaExceeded' in str(e): raise e # Relanzar
            logging.error(f"    Error HTTP obteniendo likes para lote ({len(batch_ids)} IDs): {e}")
            # Podríamos intentar continuar con el siguiente lote, pero es probable que falle también
            break # Salir del bucle de lotes si falla uno
        except Exception as e:
            logging.error(f"    Error inesperado obteniendo likes para lote: {e}")
            break # Salir del bucle de lotes

    logging.info(f"      Likes sumados de {likes_found_count}/{len(video_ids_list)} videos.")
    return total_likes

In [74]:
# Celda 9: Procesar Artistas Pendientes y Obtener Datos de YouTube
current_run_data = [] # Lista para guardar los resultados de ESTA ejecución
processed_count_this_run = 0
quota_exceeded = False

if youtube and artists_to_process:
    logging.info(f"Iniciando procesamiento de {len(artists_to_process)} artistas pendientes...")
    processing_start_time = time.time()

    for artist in artists_to_process:
        processed_count_this_run += 1
        logging.info(f"Procesando artista {processed_count_this_run}/{len(artists_to_process)}: {artist}")
        artist_info = {'artist_query': artist} # Guardar el nombre original buscado

        try:
            # 1. Buscar el canal
            channel_id = search_for_channel(youtube, artist)
            artist_info['channel_id_found'] = channel_id
            time.sleep(0.2) # Pequeña pausa después de search

            if channel_id:
                # 2. Obtener estadísticas del canal
                stats = get_channel_stats(youtube, channel_id)
                time.sleep(0.2) # Pequeña pausa después de channels.list

                if stats:
                    artist_info.update(stats)

                    # 3. Buscar los top 10 videos musicales
                    top_video_ids = search_top_videos(youtube, channel_id)
                    time.sleep(0.2) # Pequeña pausa después de search

                    # 4. Obtener y sumar likes de esos videos
                    total_top10_likes = get_video_likes(youtube, top_video_ids)
                    artist_info['total_top10_video_likes'] = total_top10_likes
                    time.sleep(0.2) # Pequeña pausa después de videos.list
                else:
                    # Si no se obtienen stats, marcar likes como no disponibles
                    artist_info.update({
                        'channel_title_verified': None, 'subscriber_count': None,
                        'view_count': None, 'video_count': None,
                        'total_top10_video_likes': None
                    })
            else:
                # Si no se encuentra canal, rellenar todo
                artist_info.update({
                    'channel_title_verified': None, 'subscriber_count': None,
                    'view_count': None, 'video_count': None,
                    'total_top10_video_likes': None
                })

            current_run_data.append(artist_info)

        except googleapiclient.errors.HttpError as e:
            if 'quotaExceeded' in str(e):
                logging.error(f"¡CUOTA EXCEDIDA procesando a {artist}! Deteniendo esta ejecución.")
                quota_exceeded = True
                # Asegurarse de añadir la info parcial del artista actual si se desea
                if 'channel_id_found' not in artist_info: artist_info['channel_id_found'] = None
                if 'channel_title_verified' not in artist_info: artist_info['channel_title_verified'] = None
                if 'subscriber_count' not in artist_info: artist_info['subscriber_count'] = None
                if 'view_count' not in artist_info: artist_info['view_count'] = None
                if 'video_count' not in artist_info: artist_info['video_count'] = None
                if 'total_top10_video_likes' not in artist_info: artist_info['total_top10_video_likes'] = None
                current_run_data.append(artist_info) # Añadir info parcial antes de salir
                break # Salir del bucle for
            else:
                logging.error(f"Error HTTP inesperado procesando {artist}: {e}")
                # Añadir info parcial con error
                if 'channel_id_found' not in artist_info: artist_info['channel_id_found'] = f"Error: {e}"
                current_run_data.append(artist_info)

        except Exception as e:
             logging.error(f"Error general inesperado procesando {artist}: {e}")
             # Añadir info parcial con error
             if 'channel_id_found' not in artist_info: artist_info['channel_id_found'] = f"Error: {e}"
             current_run_data.append(artist_info)

        # Pausa general para evitar exceder cuota (ajustable)
        time.sleep(1)

    processing_end_time = time.time()
    logging.info(f"Procesamiento de {processed_count_this_run} artistas finalizado en {processing_end_time - processing_start_time:.2f} segundos.")

elif not youtube:
    logging.error("El objeto de servicio de YouTube no está disponible (falló la autenticación).")
elif not artists_to_process:
    logging.info("No hay artistas pendientes para procesar.")
else:
     logging.warning("No se procesaron artistas por una razón desconocida.")

2025-04-11 02:56:01,044 - INFO - Iniciando procesamiento de 32593 artistas pendientes...
2025-04-11 02:56:01,045 - INFO - Procesando artista 1/32593: Foster;Chelsea Collins
2025-04-11 02:56:01,045 - INFO -   Buscando canal para: 'Foster;Chelsea Collins' (Query: 'Foster;Chelsea Collins Topic')...
2025-04-11 02:56:01,551 - WARNING - Encountered 403 Forbidden with reason "quotaExceeded"
2025-04-11 02:56:01,552 - ERROR - ¡CUOTA EXCEDIDA procesando a Foster;Chelsea Collins! Deteniendo esta ejecución.
2025-04-11 02:56:01,553 - INFO - Procesamiento de 1 artistas finalizado en 0.51 segundos.


In [75]:
# Celda 10: Combinar Resultados y Guardar en CSV
df_combined_results = pd.DataFrame()

if current_run_data: # Si se procesó al menos un artista en esta ejecución
    df_current_run = pd.DataFrame(current_run_data)
    logging.info(f"Se creará/actualizará el CSV con {len(df_current_run)} nuevos registros procesados.")
    # Concatenar con los datos previamente procesados (si los había)
    df_combined_results = pd.concat([df_processed_so_far, df_current_run], ignore_index=True)
    # Eliminar posibles duplicados por si se re-procesó alguno por error
    df_combined_results.drop_duplicates(subset=['artist_query'], keep='last', inplace=True)

elif not df_processed_so_far.empty:
     logging.info("No se procesaron nuevos artistas en esta ejecución. Se guardará el estado anterior.")
     df_combined_results = df_processed_so_far # Mantener los datos anteriores
else:
     logging.info("No se procesaron artistas en esta ejecución y no había datos previos.")


# Guardar el DataFrame combinado (sobrescribiendo el archivo)
if not df_combined_results.empty:
    try:
        logging.info(f"Guardando {len(df_combined_results)} registros combinados en {PROGRESS_CSV_PATH}...")
        df_combined_results.to_csv(PROGRESS_CSV_PATH, index=False)
        logging.info(f"Resultados combinados guardados exitosamente.")
        print("\n--- Primeras filas del archivo de resultados guardado ---")
        display(df_combined_results.head())
        print("\n--- Últimas filas del archivo de resultados guardado ---")
        display(df_combined_results.tail())
    except Exception as e:
        logging.error(f"Error al guardar los resultados combinados en CSV: {e}")
elif quota_exceeded:
    logging.warning("La cuota fue excedida y no se generaron nuevos datos para guardar.")
else:
    logging.info("No hay resultados para guardar (ni previos ni de esta ejecución).")


logging.info("--- Script de Enriquecimiento con YouTube API Finalizado ---")

2025-04-11 02:56:01,560 - INFO - Se creará/actualizará el CSV con 1 nuevos registros procesados.
2025-04-11 02:56:01,562 - INFO - Guardando 98 registros combinados en /home/nicolas/Escritorio/workshops/workshop_2/data/youtube_stats.csv...
2025-04-11 02:56:01,564 - INFO - Resultados combinados guardados exitosamente.



--- Primeras filas del archivo de resultados guardado ---


,artist_query,channel_id_found,channel_title_verified,subscriber_count,view_count,video_count,total_top10_video_likes
0,Nalan,UC_zzCBiTkpQwP8lwHgQ7M3Q,Nalan - Topic,19400.0,5.372364e+07,187.0,163164.0
1,Grupo Sensação,UCKiMawhTZ5z1S8i1_fezSAw,Grupo Sensação,55800.0,8.161696e+06,24.0,0.0
2,Gorillaz;Beck,UCNIV5B_aJnLrKDSnW_MOmcQ,Gorillaz - Topic,43100.0,1.533592e+09,560.0,9354631.0
3,Parcels,UC2as7PrmUgmdZAkMIWNY6EQ,Parcels,306000.0,1.803548e+08,236.0,745054.0
4,Klingande,UCOX8OMkI7ULP7K8bfB_HTHA,Klingande,83500.0,6.479854e+07,44.0,163822.0



--- Últimas filas del archivo de resultados guardado ---


,artist_query,channel_id_found,channel_title_verified,subscriber_count,view_count,video_count,total_top10_video_likes
93,Sachet Tandon;The Fusion Project,UCWacu_XCRrReAebduGSrGJQ,The Fusion Project - Topic,85.0,3297.0,14.0,40.0
94,Too Much Joy,UCaS6hAeJPwHqSpp9Q3stclg,Too Much Joy - Topic,434.0,210069.0,226.0,992.0
95,張偉文;胡美儀,UCjp4hbsLF7OxCJrgWmE9CWQ,粤调粿粿,19.0,8777.0,39.0,0.0
96,Daniel Adams-Ray;Erik Lundin,NaN,NaN,NaN,NaN,NaN,NaN
97,Foster;Chelsea Collins,None,None,NaN,NaN,NaN,NaN


2025-04-11 02:56:01,575 - INFO - --- Script de Enriquecimiento con YouTube API Finalizado ---
